## It happens all the time: someone gives you data containing malformed strings,Python lists and missing data. 
## How do you tidy it up so you can get on with the analysis?
## Take this monstrosity as the DataFrame to use in the following puzzles:


In [1]:
import pandas as pd
import numpy as np

In [2]:
# dataframe supplied

df = pd.DataFrame(
{
    'From_To': ['LoNDon_paris', 'MAdrid_miLAN','londON_StockhOlm','Budapest_PaRis', 'Brussels_londOn'],
    'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
    'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
    'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )','12. Air France', '"Swiss Air"']
}
)
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


### 1. Some values in the the FlightNumber column are missing. These numbers are  meant to increase by 10 with each row so 10055 and 10075 need to be put in  place. Fill in these missing numbers and make the column an integer column  (instead of a float column). 

In [3]:
# flight number column

df['FlightNumber']

0    10045.0
1        NaN
2    10065.0
3        NaN
4    10085.0
Name: FlightNumber, dtype: float64

In [4]:
# get rows with null value

row_index_null = np.where(df['FlightNumber'].isnull() == True)
df.iloc[row_index_null]['FlightNumber']

1   NaN
3   NaN
Name: FlightNumber, dtype: float64

In [5]:
# update flightnumber with null value by adding 10 to the previous row flight number

for i in row_index_null[0]:
    df.at[i, 'FlightNumber'] = df.at[i-1, 'FlightNumber'] + 10

In [6]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055.0,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075.0,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


In [7]:
# convert flight number to float

df['FlightNumber'] = pd.to_numeric(df['FlightNumber'], downcast='integer')

In [8]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055,[],<Air France> (12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075,[13],12. Air France
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air"""


### 2. The From_To column would be better as two separate columns! Split each  string on the underscore delimiter _ to give a new temporary DataFrame with  the correct values. Assign the correct column names to this temporary  DataFrame. 

In [9]:
# 'From_To' column
df['From_To']

0        LoNDon_paris
1        MAdrid_miLAN
2    londON_StockhOlm
3      Budapest_PaRis
4     Brussels_londOn
Name: From_To, dtype: object

In [10]:
# split 'From_To' column to two

df2= df['From_To'].str.split("_", n = 1, expand = True)
df2

,0,1
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


In [11]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055,[],<Air France> (12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075,[13],12. Air France
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air"""


In [12]:
# update column header

df2.columns = ['From', 'To']
df2

,From,To
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


### 3. Notice how the capitalisation of the city names is all mixed up in this  temporary DataFrame. Standardise the strings so that only the first letter is  uppercase (e.g. "londON" should become "London".)

In [13]:
# update contents of 'From', 'To' column to have title case

df2['From'] = df2['From'].str.title()
df2['To'] = df2['To'].str.title()
df2

,From,To
0,London,Paris
1,Madrid,Milan
2,London,Stockholm
3,Budapest,Paris
4,Brussels,London


### 4. Delete the From_To column from df and attach the temporary DataFrame  from the previous questions.

In [14]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055,[],<Air France> (12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075,[13],12. Air France
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air"""


In [15]:
# drop 'From_To' and add 'From', 'To' column to the data frame

df.drop(['From_To'], axis = 1, inplace = True)
df['From'] = df2['From']
df['To'] = df2['To']
df


,FlightNumber,RecentDelays,Airline,From,To
0,10045,"[23, 47]",KLM(!),London,Paris
1,10055,[],<Air France> (12),Madrid,Milan
2,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,10075,[13],12. Air France,Budapest,Paris
4,10085,"[67, 32]","""Swiss Air""",Brussels,London


### 5. In the RecentDelays column, the values have been entered into the  DataFrame as a list. We would like each first value in its own column, each second value in its own column, and so on. If there isn't an Nth value, the value  should be NaN. 


In [16]:
# new df from the column of lists
df3 = pd.DataFrame(df['RecentDelays'].tolist())
# display the resulting df
df3

,0,1,2
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [17]:
# add prefix to header of columns

df3 = df3.add_prefix('delay_')

In [18]:
df3

,delay_0,delay_1,delay_2
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [19]:
df

,FlightNumber,RecentDelays,Airline,From,To
0,10045,"[23, 47]",KLM(!),London,Paris
1,10055,[],<Air France> (12),Madrid,Milan
2,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,10075,[13],12. Air France,Budapest,Paris
4,10085,"[67, 32]","""Swiss Air""",Brussels,London


In [20]:
# concat delay dataframes and main dataframe

df = pd.concat([df, df3], axis=1)
df

,FlightNumber,RecentDelays,Airline,From,To,delay_0,delay_1,delay_2
0,10045,"[23, 47]",KLM(!),London,Paris,23.0,47.0,NaN
1,10055,[],<Air France> (12),Madrid,Milan,NaN,NaN,NaN
2,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm,24.0,43.0,87.0
3,10075,[13],12. Air France,Budapest,Paris,13.0,NaN,NaN
4,10085,"[67, 32]","""Swiss Air""",Brussels,London,67.0,32.0,NaN


In [21]:
# drop 'RecentDelays' column

df.drop(['RecentDelays'], axis = 1, inplace = True)
df

,FlightNumber,Airline,From,To,delay_0,delay_1,delay_2
0,10045,KLM(!),London,Paris,23.0,47.0,NaN
1,10055,<Air France> (12),Madrid,Milan,NaN,NaN,NaN
2,10065,(British Airways. ),London,Stockholm,24.0,43.0,87.0
3,10075,12. Air France,Budapest,Paris,13.0,NaN,NaN
4,10085,"""Swiss Air""",Brussels,London,67.0,32.0,NaN


## Final output as per document

In [22]:
df

,FlightNumber,Airline,From,To,delay_0,delay_1,delay_2
0,10045,KLM(!),London,Paris,23.0,47.0,NaN
1,10055,<Air France> (12),Madrid,Milan,NaN,NaN,NaN
2,10065,(British Airways. ),London,Stockholm,24.0,43.0,87.0
3,10075,12. Air France,Budapest,Paris,13.0,NaN,NaN
4,10085,"""Swiss Air""",Brussels,London,67.0,32.0,NaN


### data cleaning operation performed on Airline column(not given in document)

In [23]:
# remove unnecessary punctuations and numbers from Airline column

import string 

def remove_punctuations(text):
    airline = ""
    for literal in text:
        if literal in string.ascii_letters:
            airline += literal
    return airline
# Apply to the DF series
df['Airline'] = df['Airline'].apply(remove_punctuations)
df

,FlightNumber,Airline,From,To,delay_0,delay_1,delay_2
0,10045,KLM,London,Paris,23.0,47.0,NaN
1,10055,AirFrance,Madrid,Milan,NaN,NaN,NaN
2,10065,BritishAirways,London,Stockholm,24.0,43.0,87.0
3,10075,AirFrance,Budapest,Paris,13.0,NaN,NaN
4,10085,SwissAir,Brussels,London,67.0,32.0,NaN
